# Georeferenciamento da base de dados de IPTU do Município da Cidade de São Paulo

Está disponível para download através do site do [Geosampa](http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/_SBC.aspx) os arquivos de IPTU desde 1995 até agora. Esses arquivos contem registros de todos os lancamentos de IPTU além de informações que podem ser bastante úteis para compreender a cidade, seus parcelamentos e edificações.

## Objetivo

Com a finalidade de facilitar o acesso e estimular de maneira pedagógica a análise dos dados abertos disponíveis criamos aqui um passo-a-passo de como carregar as informações do IPTU, fazer algumas análises básicas e posteriormente georeferenciar essas informações.

## Metodologia

Para isso vamos utilizar a linguagem de programação Python e utilizar a biblioteca de analise de dados Pandas e GeoPandas, e exemplificar passo-a-passo como acessar, analisar e georeferenciar os dados do IPTU

## Download da Tabela de IPTU

A [Tabela de IPTU de 2019 está disponível nesse link](mapas.geosampa.prodam/PaginasPublicas/downloadIfr.aspx?orig=DownloadCamadas&arq=11_Cadastro%5C%5CIPTU%5C%5CXLS_CSV%5C%5CIPTU2019&arqTipo=XLS_CSV), é disponibilizada em formato CSV e é disponibilizada em um arquivo ZIP.

Vamos expandir o arquivo ZIP na pasta Downloads e carregar o arquivo na memória com o Pandas

In [14]:
%%time

import pandas as pd

df = pd.read_csv("./downloads/IPTU2019.CSV", error_bad_lines=False, encoding='iso-8859-9', sep=';')
df.head()

Wall time: 31.3 s


,NUMERO DO CONTRIBUINTE,ANO DO EXERCICIO,NUMERO DA NL,DATA DO CADASTRAMENTO,TIPO DE CONTRIBUINTE 1,CPF/CNPJ DO CONTRIBUINTE 1,NOME DO CONTRIBUINTE 1,TIPO DE CONTRIBUINTE 2,CPF/CNPJ DO CONTRIBUINTE 2,NOME DO CONTRIBUINTE 2,...,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,TIPO DE TERRENO,FATOR DE OBSOLESCENCIA,ANO DE INICIO DA VIDA DO CONTRIBUINTE,MES DE INICIO DA VIDA DO CONTRIBUINTE,FASE DO CONTRIBUINTE
0,0010030001-4,2019,1,12/01/19,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,1924,1,"13,00",Loja,Comercial horizontal - padrão B,De esquina,"0,20",1963,1,0
1,0010030002-2,2019,1,12/01/19,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,1944,1,"6,00",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0
2,0010030003-0,2019,1,12/01/19,PESSOA FISICA (CPF),XXXXXX0214XXXX,MARCIO MOURCHED,NaN,,NaN,...,1965,2,"7,85",Loja,Comercial horizontal - padrão B,Normal,"0,32",1963,1,0
3,0010030004-9,2019,1,12/01/19,PESSOA FISICA (CPF),XXXXXX2094XXXX,AUGUSTO CESAR DE MATTOS JUNIOR,NaN,,NaN,...,1944,1,"6,05",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0
4,0010030005-7,2019,1,12/01/19,PESSOA FISICA (CPF),XXXXXX2094XXXX,AUGUSTO CESAR DE MATTOS JUNIOR,NaN,,NaN,...,1944,1,"6,70",Loja,Comercial horizontal - padrão B,Normal,"0,20",1963,1,0


Temos 35 colunas na base do IPTU conforme a lista a seguir:

In [15]:
list(df.columns)

['NUMERO DO CONTRIBUINTE',
 'ANO DO EXERCICIO',
 'NUMERO DA NL',
 'DATA DO CADASTRAMENTO',
 'TIPO DE CONTRIBUINTE 1',
 'CPF/CNPJ DO CONTRIBUINTE 1',
 'NOME DO CONTRIBUINTE 1',
 'TIPO DE CONTRIBUINTE 2',
 'CPF/CNPJ DO CONTRIBUINTE 2',
 'NOME DO CONTRIBUINTE 2',
 'NUMERO DO CONDOMINIO',
 'CODLOG DO IMOVEL',
 'NOME DE LOGRADOURO DO IMOVEL',
 'NUMERO DO IMOVEL',
 'COMPLEMENTO DO IMOVEL',
 'BAIRRO DO IMOVEL',
 'REFERENCIA DO IMOVEL',
 'CEP DO IMOVEL',
 'QUANTIDADE DE ESQUINAS/FRENTES',
 'FRACAO IDEAL',
 'AREA DO TERRENO',
 'AREA CONSTRUIDA',
 'AREA OCUPADA',
 'VALOR DO M2 DO TERRENO',
 'VALOR DO M2 DE CONSTRUCAO',
 'ANO DA CONSTRUCAO CORRIGIDO',
 'QUANTIDADE DE PAVIMENTOS',
 'TESTADA PARA CALCULO',
 'TIPO DE USO DO IMOVEL',
 'TIPO DE PADRAO DA CONSTRUCAO',
 'TIPO DE TERRENO',
 'FATOR DE OBSOLESCENCIA',
 'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
 'MES DE INICIO DA VIDA DO CONTRIBUINTE',
 'FASE DO CONTRIBUINTE']

Isso abre uma série de possibilidades de análises, sejam elas individuais ou combinadas, como por exemplo pesquisar a quantidade de construções da década de 1970

In [16]:
%%time
df_decada_70 = df[df['ANO DA CONSTRUCAO CORRIGIDO'].between(1970, 1980)].index

print(len(df_decada_70)) # Total de unidades/lancamentos construidas na década de 1970
print(len(df)) # Total de unidades/lancamentos total

print(len(df_decada_70)/len(df)) # Percentual de unidades/lancamentos produzidos na década de 1970

828888
3447112
0.24045867961354317
Wall time: 485 ms


Na máquina que está realizando esses testes a performance não está restringindo as análises. O arquivo demora cerca de 34 segundos para carregar e a análise acima cerca de 359ms.

## Removendo o dígito do SQL

Algumas vezes pode ser interessante ter o SQL sem o dígito para fazer alguma consulta, para isso vamos criar uma possibilidade

In [17]:
%%time
df['NUMERO DO CONTRIBUINTE'] = df['NUMERO DO CONTRIBUINTE'].apply(lambda x: x.split('-')[0])

Wall time: 1.36 s


In [18]:
df.to_csv("./downloads/IPTU2019-SQL.CSV", index = None, header=True)